In [7]:
import pyspark
from datetime import datetime, timedelta


spark = pyspark.sql.SparkSession.builder.appName("auctions").getOrCreate()
df_auctions = spark.read.csv('auctions.csv', header=True)
df_installs = spark.read.csv('installs.csv', header=True)
rdd_auctions = df_auctions.rdd
rdd_installs = df_installs.rdd


In [8]:
rdd_auctions.take(5)

[Row(date=u'2019-04-23 18:58:00.842116', device_id=u'2564673204772915246', ref_type_id=u'1', source_id=u'0'),
 Row(date=u'2019-04-23 18:58:01.530771', device_id=u'4441121667607578179', ref_type_id=u'7', source_id=u'0'),
 Row(date=u'2019-04-23 18:58:01.767562', device_id=u'7721769811471055264', ref_type_id=u'1', source_id=u'0'),
 Row(date=u'2019-04-23 18:58:02.363468', device_id=u'6416039086842158968', ref_type_id=u'1', source_id=u'0'),
 Row(date=u'2019-04-23 18:58:02.397559', device_id=u'1258642015983312729', ref_type_id=u'1', source_id=u'0')]

In [ ]:
rdd_auctions1 = rdd_auctions.map(lambda x: (x[1], datetime.strptime(x[0], "%Y-%m-%d %H:%M:%S.%f")))

In [ ]:
rdd_auctions2 = rdd_auctions1.map(lambda x: (x[0], x[1], x[1].strftime("%H"), x[1].strftime("%M")))

In [ ]:
def getRange(row):
    a = None
    for i in range(0,6):
        if i*10<=int(row[3]) and int(row[3])<=(i+1)*10:
            a = i
    return (row[0], row[1], row[2], row[3], a)

In [ ]:
rdd_auctions3 = rdd_auctions2.map(getRange)

In [ ]:
rdd_auctions3.take(1)

In [ ]:
rdd_auctions3

In [17]:
def toCSVLine(data):
    try:
        return ','.join(str(d)) for d in data)
    except:
        return ','.join('' for d in data)

In [ ]:
lines_train = rdd_auctions3.map(toCSVLine)
rdd_auctions3.repartition(1).saveAsTextFile('auction_with_range.csv')

In [ ]:
# Installs

In [9]:
df_installs = spark.read.csv('installs.csv', header=True)
rdd_installs = df_installs.rdd

In [10]:
rdd_installs1 = rdd_installs.map(lambda x: (datetime.strptime(x[0], "%Y-%m-%d %H:%M:%S.%f"), \
                                  x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12], x[13],\
                                           x[14], x[15], x[16], x[17]))

In [11]:
rdd_installs.take(1)

[Row(created=u'2019-04-24 06:23:29.495', application_id=u'1', ref_type=u'1494519392962156891', ref_hash=u'4716708407362582887', click_hash=None, attributed=u'False', implicit=u'True', device_countrycode=u'6287817205707153877', device_brand=None, device_model=u'3.739127126472163e+17', session_user_agent=u'adjust.com', user_agent=None, event_uuid=u'79837499-2f2a-4605-a663-e322f759424f', kind=u'app_open', wifi=None, trans_id=None, ip_address=u'4243443387795468703', device_language=u'3.3013777759776993e+18')]

In [12]:
rdd_installs2 = rdd_installs1.map(lambda x: (x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12], x[13],\
                                           x[14], x[15], x[16], x[17],x[0].strftime("%H"), x[0].strftime("%M")))

In [13]:
def getRange(row):
    a = None
    for i in range(0,6):
        if i*10<=int(row[19]) and int(row[19])<=(i+1)*10:
            a = i
    return (row[0], row[1], row[2], row[3], row[4], row[5],\
            row[6], row[7], row[8], row[9], row[10], row[11], row[12],\
            row[13], row[14], row[15], row[16], row[17], row[18], row[19], a)

In [14]:
rdd_installs3 = rdd_installs2.map(getRange)

In [16]:
rdd_installs3.take(1)

[(datetime.datetime(2019, 4, 24, 6, 23, 29, 495000),
  u'1',
  u'1494519392962156891',
  u'4716708407362582887',
  None,
  u'False',
  u'True',
  u'6287817205707153877',
  None,
  u'3.739127126472163e+17',
  u'adjust.com',
  None,
  u'79837499-2f2a-4605-a663-e322f759424f',
  u'app_open',
  None,
  None,
  u'4243443387795468703',
  u'3.3013777759776993e+18',
  '06',
  '23',
  2)]

In [ ]:
def toCSVLine(data):
    try:
        return ','.join(str(d)) for d in data)
    except:
        return ','.join('' for d in data)

In [18]:
rdd_installs4 = rdd_installs3.map(toCSVLine)
rdd_installs4.repartition(1).saveAsTextFile('installs_with_range.csv')

In [19]:
import pandas as pd

In [22]:
installs_range = pd.read_csv('installs_with_range.csv', engine='python')

IOError: [Errno 21] Is a directory: 'installs_with_range.csv'